In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vvvvvvvvvvvvv/jd_ner_model_pickle.pkl
/kaggle/input/vvvvvvvvvvvvv/elecengineer.pdf
/kaggle/input/vvvvvvvvvvvvv/resume_ner_model_pickle.pkl


Installing all the necesarry modules

In [2]:
!pip install simpletransformers
!pip install PyMuPDF
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 78.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.5 MB/s eta 0:00:00:00:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a4658f4009c4e85553cca44ed45251acd17b98a03050cc1e8200ac3fe76ba199
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 67.3 MB/s eta 0:00:00:00:0100:01


Importing all the necesarry Libraries

In [3]:
import spacy
import fitz
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from typing import List, Dict
from collections import Counter
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from collections import defaultdict 
from transformers import AutoConfig

Loading the Job Description Parser Model

In [4]:
file_size = os.path.getsize('/kaggle/input/vvvvvvvvvvvvv/jd_ner_model_pickle.pkl')
with open('/kaggle/input/vvvvvvvvvvvvv/jd_ner_model_pickle.pkl', 'rb') as file:
  resume_model = pickle.load(file) 
print(f"Model loaded successfully! File size: {file_size} bytes")  


Model loaded successfully! File size: 1330618217 bytes


Loading the Resume Parser Model

In [5]:
file_size = os.path.getsize('/kaggle/input/vvvvvvvvvvvvv/resume_ner_model_pickle.pkl')
with open('/kaggle/input/vvvvvvvvvvvvv/resume_ner_model_pickle.pkl', 'rb') as file:
  jd_model = pickle.load(file)

print(f"Model loaded successfully! File size: {file_size} bytes")  


Model loaded successfully! File size: 1330560118 bytes


Defining the Text Preprocessing and pdf  loading functions


In [6]:

nlp=spacy.load("en_core_web_sm")

def text_preprocess(yo):
 doc=nlp(yo)

 string=[]

 for token in doc:
  if token.is_stop==False and token.is_punct==False: # Removing stop words and removing punctuation marks
     string.append(token.lemma_)  #Lemmatization

 return " ".join(string)

In [7]:
def pdf_load(name):
  pdf_file = name
  doc = fitz.open(pdf_file)

  text = " "

  for page_num in range(doc.page_count):
      page = doc.load_page(page_num)
      text += page.get_text()

  return (" ".join(text.split("\n")))

Functions to get predictions of annotations of resume and jds

In [8]:
class EnsembleNERResume:
    def initialize(self, pretrained_model_path, device='cuda'):
       
        config = AutoConfig.from_pretrained('bert-large-cased')  

       
        self.bert_model = AutoModelForTokenClassification.from_config(config) 

      
        with open(pretrained_model_path, 'rb') as file:
       
            self.bert_model = pickle.load(file)  #loading the pickle model
         

        self.bert_tokenizer = AutoTokenizer.from_pretrained('bert-large-cased')

        # Device setup
        self.device = device
        self.bert_model.to(device)
        self.bert_model.eval()


    def predict(self, text: str) -> List[Dict[str, str]]:
        # Tokenize and get predictions from the BERT model
        bert_tokens = self.bert_tokenizer(
            text,
            return_tensors="pt",
            return_offsets_mapping=True,  
            padding=True,
            truncation=True
        ).to(self.device)

       
        offset_mapping = bert_tokens.pop('offset_mapping')

        with torch.no_grad():
            bert_outputs = self.bert_model(**bert_tokens)
            bert_predictions = torch.argmax(bert_outputs.logits, dim=2)[0]

        # Convert predictions to word-level labels
        word_predictions = []
        current_word = ""
        current_labels = []

        input_ids = bert_tokens["input_ids"][0]  

        for idx, (start, end) in enumerate(offset_mapping[0]):
            if start == 0 and end == 0:  
                continue

       
            token_text = self.bert_tokenizer.decode(input_ids[idx].item())
            pred_label = self.bert_model.config.id2label[bert_predictions[idx].item()]

           
            if token_text.startswith("##"):
                current_word += token_text[2:]  
                current_labels.append(pred_label)
            else:
                if current_word:
                   
                    final_label = Counter(current_labels).most_common(1)[0][0]
                    word_predictions.append({"word": current_word, "label": final_label})

               
                current_word = token_text
                current_labels = [pred_label]

       
        if current_word and current_labels:
            final_label = Counter(current_labels).most_common(1)[0][0]
            word_predictions.append({"word": current_word, "label": final_label}) 

        return word_predictions

In [9]:
class EnsembleNERJd:
    def initialize(self, pretrained_model_path, device='cuda'):
       
        config = AutoConfig.from_pretrained('bert-large-cased')  

      
        self.bert_model = AutoModelForTokenClassification.from_config(config) # loding the pickle model

       
        with open(pretrained_model_path, 'rb') as file:
            
            self.bert_model = pickle.load(file)  
            

        self.bert_tokenizer = AutoTokenizer.from_pretrained('bert-large-cased')

        # Device setup
        self.device = device
        self.bert_model.to(device)
        self.bert_model.eval()


    def predict(self, text: str) -> List[Dict[str, str]]:
     
        bert_tokens = self.bert_tokenizer(
            text,
            return_tensors="pt",
            return_offsets_mapping=True,  
            padding=True,
            truncation=True
        ).to(self.device)

        
        offset_mapping = bert_tokens.pop('offset_mapping')

        with torch.no_grad():
            bert_outputs = self.bert_model(**bert_tokens)
            bert_predictions = torch.argmax(bert_outputs.logits, dim=2)[0]

        # Convert predictions to word-level labels
        word_predictions = []
        current_word = ""
        current_labels = []

        input_ids = bert_tokens["input_ids"][0]  

        for idx, (start, end) in enumerate(offset_mapping[0]):
            if start == 0 and end == 0:  
                continue

           
            token_text = self.bert_tokenizer.decode(input_ids[idx].item())
            pred_label = self.bert_model.config.id2label[bert_predictions[idx].item()]

          
            if token_text.startswith("##"):
                current_word += token_text[2:]  
                current_labels.append(pred_label)
            else:
                if current_word:
                    
                    final_label = Counter(current_labels).most_common(1)[0][0]
                    word_predictions.append({"word": current_word, "label": final_label}) 

               
                current_word = token_text
                current_labels = [pred_label]

       
        if current_word and current_labels:
            final_label = Counter(current_labels).most_common(1)[0][0]
            word_predictions.append({"word": current_word, "label": final_label})

        return word_predictions

Functions to convert the prediction in JSON format

In [10]:
def group_entities_unique_jd(predictions: List[Dict[str, str]]) -> Dict[str, list]:
    entities = defaultdict(set)  # Use a set to store unique entities
    current_entity = []
    current_label = None

    for pred in predictions:
        word = pred['word']
        label = pred['label'].strip() 

        if label.startswith("B-"):
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity))  
            current_label = label[2:]  
            current_entity = [word]  
        elif label.startswith("I-") and current_label == label[2:]:
            current_entity.append(word)  
        elif label.startswith("I-") and current_label != label[2:]:
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity))  
            current_label = label[2:]  
            current_entity = [word]
        else:
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity))  
            current_entity = []
            current_label = None

    if current_entity and current_label:
        entities[current_label].add(" ".join(current_entity)) 

    # Convert sets back to lists for JSON 
    return {label: list(entity_set) for label, entity_set in entities.items()}





In [11]:
def group_entities_unique_resume(predictions: List[Dict[str, str]]) -> Dict[str, list]:

    entities = defaultdict(set)  # Use a set to store unique entities
    current_entity = []
    current_label = None

    for pred in predictions:
        word = pred['word']
        label = pred['label']

        if label.startswith("B-"): 
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity))  
            current_label = label[2:]  
            current_entity = [word]
        elif label.startswith("I-") and current_label == label[2:]: 
            current_entity.append(word)
        else: 
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity)) 
            current_entity = []
            current_label = None

    if current_entity and current_label:
        entities[current_label].add(" ".join(current_entity))  
    # Convert sets back to lists for JSON 
    return {label: list(entity_set) for label, entity_set in entities.items()}




Below few codes loads the pdfs of various resumes and get the annotated resumes

In [69]:
text1=text_preprocess(pdf_load("/kaggle/input/vvvvvvvvvvvvv/elecengineer.pdf"))
ensemble = EnsembleNERResume(
        pretrained_model_path="/kaggle/input/vvvvvvvvvvvvv/resume_ner_model_pickle.pkl",
        device="cuda" if torch.cuda.is_available() else "cpu")

  
predictions1 = ensemble.predict(text1)

In [70]:
text2=text_preprocess(pdf_load("/kaggle/input/resumes/animator.pdf"))
ensemble = EnsembleNERResume(
        pretrained_model_path="/kaggle/input/vvvvvvvvvvvvv/resume_ner_model_pickle.pkl",
        device="cuda" if torch.cuda.is_available() else "cpu")

  
predictions2 = ensemble.predict(text2)

In [71]:

text3=text_preprocess(pdf_load("/kaggle/input/resumes/bartender.pdf"))
ensemble = EnsembleNERResume(
        pretrained_model_path="/kaggle/input/vvvvvvvvvvvvv/resume_ner_model_pickle.pkl",
        device="cuda" if torch.cuda.is_available() else "cpu")

  
predictions3 = ensemble.predict(text3)

In [80]:
elecengresume=group_entities_unique_resume(predictions1)


In [81]:
animatorresume=group_entities_unique_resume(predictions2)


In [82]:
bartenderresume=group_entities_unique_resume(predictions3)


Parsing and getting the annotations for JOB Description

In [58]:
jd=text_preprocess("Electrical Engineer,Electronics Hardware Engineers develop and design electronic components and systems, from circuit boards to hardware prototypes, ensuring functionality and performance.")
ensemble = EnsembleNERJd(
        pretrained_model_path="/kaggle/input/vvvvvvvvvvvvv/jd_ner_model_pickle.pkl",
        device="cuda" if torch.cuda.is_available() else "cpu")

  
predictionsjd = ensemble.predict(jd)

In [83]:
elecengjd=group_entities_unique_jd(predictionsjd)

Loading the Pre trained T5 model

In [15]:
# Load the Sentence-BERT model
bert_model = SentenceTransformer('sentence-t5-xxl')  

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Defing Function to Calculate Cosine Similarity between Various Resumes and Job Descriptions

In [84]:
def eval(res,jd):

 resume = res# resume dictionary
 job_description =jd  #  job description dictionary

# Preprocess and join fields for embedding generation
 def preprocess_field(field):
     return " ".join(field) if field else ""

# Define the fields and weights
 fields = {
    'Skills': {'resume_key': 'Skills', 'jd_key': 'Skill', 'weight': 0.3},
    'Degree': {'resume_key': 'Degree', 'jd_key': 'Degree', 'weight': 0.2},
    'Designation': {'resume_key': 'Designation', 'jd_key': 'Designation', 'weight': 0.2},
    'Experience': {'resume_key': 'Experience', 'jd_key': 'Experience', 'weight': 0.2},
    'Companies Worked At': {'resume_key': 'Companies Worked At', 'jd_key': 'Companies Worked At', 'weight': 0.1},
 }


 total_similarity = 0
 total_weight = 0

# Process each field
 for field_name, field_info in fields.items():
   
    resume_field = resume.get(field_info['resume_key'], [])
    jd_field = job_description.get(field_info['jd_key'], [])
    
   
    if resume_field and jd_field:
        
        resume_text = preprocess_field(resume_field)
        jd_text = preprocess_field(jd_field)
        
        # Generate embeddings using BERT
        resume_vector = bert_model.encode(resume_text)
        jd_vector = bert_model.encode(jd_text)
        
       
        similarity = cosine_similarity([resume_vector], [jd_vector])[0][0]
    else:
        
        similarity = 0
    
    
    weighted_similarity = field_info['weight'] * similarity
    total_similarity += weighted_similarity
    total_weight += field_info['weight']
    
    
    print(f"{field_name} Similarity: {similarity:.4f} (Weighted Contribution: {weighted_similarity:.4f})")


 overall_similarity = total_similarity / total_weight if total_weight > 0 else 0


 print(f"Overall Similarity: {overall_similarity:.4f}")


In [85]:
eval(elecengresume,elecengjd)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Skills Similarity: 0.7764 (Weighted Contribution: 0.2329)
Degree Similarity: 0.0000 (Weighted Contribution: 0.0000)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Designation Similarity: 0.8746 (Weighted Contribution: 0.1749)
Experience Similarity: 0.0000 (Weighted Contribution: 0.0000)
Companies Worked At Similarity: 0.0000 (Weighted Contribution: 0.0000)
Overall Similarity: 0.4078


In [86]:
eval(animatorresume,elecengjd)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Skills Similarity: 0.6904 (Weighted Contribution: 0.2071)
Degree Similarity: 0.0000 (Weighted Contribution: 0.0000)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Designation Similarity: 0.7811 (Weighted Contribution: 0.1562)
Experience Similarity: 0.0000 (Weighted Contribution: 0.0000)
Companies Worked At Similarity: 0.0000 (Weighted Contribution: 0.0000)
Overall Similarity: 0.3633


In [87]:
eval(bartenderresume,elecengjd)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Skills Similarity: 0.6801 (Weighted Contribution: 0.2040)
Degree Similarity: 0.0000 (Weighted Contribution: 0.0000)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Designation Similarity: 0.7328 (Weighted Contribution: 0.1466)
Experience Similarity: 0.0000 (Weighted Contribution: 0.0000)
Companies Worked At Similarity: 0.0000 (Weighted Contribution: 0.0000)
Overall Similarity: 0.3506
